In [ ]:
%ls

In [1]:
from pyvirtualdisplay import Display
from pyvirtualdisplay import Display
import torch
import open3d as o3d

from arguments import get_config, load_config
from interfaces import setup_LMP, LMP_interface
from visualizers import ValueMapVisualizer
from utils import set_lmp_objects
import numpy as np
import engine_interfaces
from envs.pyrep_quad_env import VoxPoserPyRepQuadcopterEnv
from engine_interfaces import Dummy
from VLMPipline.VLM import VLM

torch.set_grad_enabled(False)
disp = Display(visible=False, size=(1920, 1080))
disp.start()
# vlm config
owlv2_model_path = "/models/google-owlv2-large-patch14-finetuned"
owlv2_model_path = "/models/google-owlv2-base-patch16-ensemble"
sam_model_path = "/models/facebook-sam-vit-huge"
# sam_model_path = "/models/facebook-sam-vit-base"
xmem_model_path = "/models/XMem.pth"
resnet_18_path = "/models/resnet18.pth"
resnet_50_path = "/models/resnet50.pth"
config_path = "configs/pyrep_quadcopter.yaml"
scene_path = "./scene/quadcopter.ttt"
config = get_config(config_path=config_path)
vlmpipeline = VLM(
    owlv2_model_path,
    sam_model_path,
    xmem_model_path,
    resnet_18_path,
    resnet_50_path,
    verbose=True,
    resize_to=[640,640],
    verbose_frame_every=1,
)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using GPU
Using GPU
Hyperparameters read from the model weights: C^k=64, C^v=512, C^h=64
Single object mode: False


In [2]:


visualizer = ValueMapVisualizer(config['visualizer'])
env = VoxPoserPyRepQuadcopterEnv(visualizer=visualizer,headless=True,coppelia_scene_path=scene_path,vlmpipeline=vlmpipeline)
descriptions, obs = env.reset()
dummy_engine = Dummy()
lmps, lmp_env = setup_LMP(env, config, debug=False, engine_call_fn=dummy_engine.__call__)
voxposer_ui = lmps['plan_ui']
set_lmp_objects(lmps, env.get_object_names())

voxposer_ui(descriptions)


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


Camera parameters: {'cam_NE': {'extrinsic_params': array([[ 7.07106709e-01,  0.00000000e+00, -7.07106888e-01,
         4.80000019e+00],
       [-7.07106888e-01, -1.19209290e-07, -7.07106829e-01,
         4.80000019e+00],
       [ 0.00000000e+00,  1.00000012e+00, -1.19209290e-07,
         1.50000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), 'intrinsic_params': array([[-659.3946015,    0.       ,  240.       ],
       [   0.       , -659.3946015,  240.       ],
       [   0.       ,    0.       ,    1.       ]]), 'far_near': (20.0, 0.009999999776482582)}, 'cam_NW': {'extrinsic_params': array([[ 7.07106709e-01,  0.00000000e+00,  7.07106888e-01,
        -4.80000019e+00],
       [ 7.07106888e-01, -1.19209290e-07, -7.07106829e-01,
         4.80000019e+00],
       [ 0.00000000e+00,  1.00000012e+00, -1.19209290e-07,
         1.50000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), 'intrinsic_param

/mnt/workspace/src/VLMPipline/VLM.py:83: UserWarning: the frame shape has been changed by resizing, (3, 480, 480) -> (3, 640, 640), so we will resize the finnal mask to the original shape, which may cause some problems.
  warnings.warn(f"the frame shape has been changed by resizing, {self.original_frame_shape} -> {frame.shape}, so we will resize the finnal mask to the original shape, which may cause some problems.")


In [ ]:
env._pyrep.stop()
env._pyrep.shutdown()